# WorkShop_02

## Immagine di riferimento

<img src = "./esempioStruttura.jpg">

In [19]:
"""ggpl_bone_structure è una funzione che dato un file .csv del tipo:

0,0,0
1.,1.|1.,1.|3.,5.,3.,5.,3.,5.,3.,5.,3.|6.,3.,6.,3.,6.,3.,6.,3.
6.0,0,0
1.,1.|1.,1.|3.,5.,3.,5.,3.,5.,3.,5.,3.|6.,3.,6.,3.,6.,3.,6.,3.

in cui:
- le righe dispari indicano il vettore 3D che individua l'origine del telaio successivo, rispetto a quello precedente
- le righe pari, separate da "|", indicano tuple e liste necessarie alla costruzione del telaio tramite la funzione del
workshop precedente.

restituisce la struttura 3D completa di un edificio in cemento armato.
"""

In [12]:
from larlib import *
import csv

def ggpl_bone_structure(file_name):
    
    """funzione del workshop_01
    createFrame è una funzione che dati:

    @pillarXY = (px,py) -> sezione di un pilastro sul piano XY 
    @beamXZ = (bx,bz) -> sezione di una trave sul piano XZ
    @pillarDis = [dx1,dx2 ...] = lista delle distanze tra assi dei pilastri per ogni coppia di pilastri
    @beamDis = [hz1, hz2 ...] = lista delle distanze fra le parti superiori delle travi per ogni coppiadi travi
    
    restituisce un telaio in cemento armato di una parete di un edificio
    """
    
    def createFrame(pillarXY = (1.,1.),
                    beamXZ = (1.,1.),
                    pillarDis = [3.,5.,3.,5.,3.,5.,3.,5.,3.],
                    beamDis = [6.,3.,6.,3.,6.,3.,6.,3.]):
    
        """le misure delle fondamenta, centrate sull'asse del pilastro, 
        sono costruite a partire dalle misure del pilastro come segue:
        footing(fx,fy) = (px+1,py+1)
        """
        footingXY = tuple(map(lambda x:x+1, pillarXY))

        footingX = [footingXY[0]]
        footingY = [footingXY[1]]
        pillarX = [pillarXY[0]/2-footingXY[0]/2, pillarXY[0]]
        pillarY = [pillarXY[1]/2-footingXY[1]/2, pillarXY[1]]
        beamX = [beamXZ[0]/2-footingXY[0]/2, beamXZ[0]]
        beamY = [pillarXY[1]/2-footingXY[1]/2]
        
        """a partire dalle distanze tra piloni costruisco le liste di pieni/vuoti che costituiscono la
        struttura 2D sugli assi X e Y del telaio"""
        for disP in pillarDis:
            footingY.extend([footingXY[1]-disP,footingXY[1]])
            pillarY.extend([pillarXY[1]-disP,pillarXY[1]])
            beamY.extend([-pillarXY[1], disP - pillarXY[1]])

        """l'altezza delle fondamenta è fissa = 3"""
        footingZ = [3]
        
        "a partire dalla distanza fra le travi costruisco la lista di pieni/vuoti per l'asse Z"
        beamZ = [-3-2, beamXZ[1]]
        for disB in beamDis:
            beamZ.extend([beamXZ[1] - disB, beamXZ[1]])

        pillarZ = [-3,2 + SUM(beamDis) + beamXZ[1]]

        footings = INSR(PROD)([QUOTE(footingX),QUOTE(footingY),QUOTE(footingZ)])
        pillars = INSR(PROD)([QUOTE(pillarX),QUOTE(pillarY),QUOTE(pillarZ)])
        beams = INSR(PROD)([QUOTE(beamX),QUOTE(beamY),QUOTE(beamZ)])

        return STRUCT([COLOR(RED)(pillars),COLOR(GREEN)(beams),COLOR(YELLOW)(footings)])
    
    finalTraslationList = []
    beamDis = []
    pillarDis = []
    pillarDisTraslation = []
    pillarXY = ()
    beamXZ = ()
    
    """costruisco il parametro lista finalTraslationList 
    che associato a STRUCT effettuerà la traslazione del telaio della parete
    """
    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='|')
        for row in spamreader:
            if(len(row)==1):
                traslation = []
                traslationList = map(float,row[0].split(','))
                pillarDisTraslation += [traslationList[0]]
                traslation += [T([1,2,3])(traslationList)]
                finalTraslationList += traslation
            else:
                coordinateFrame = []
                for element in row:
                    frameList = map(float,element.split(','))
                    coordinateFrame += [frameList]
                pillarXY = tuple(coordinateFrame[0])
                beamXZ = tuple(coordinateFrame[1])
                pillarDis = coordinateFrame[2]
                beamDis = coordinateFrame[3]
                planeFrame = [createFrame(pillarXY,beamXZ,pillarDis,beamDis)]
                finalTraslationList += planeFrame
                
    """costruisco le travi di collegamento come fatto in precedenza, 
    utilizzando questa volta le distanze fra piloni create dalla traslazione
    """
    footingXY = tuple(map(lambda x:x+1, pillarXY))
    beamX = [pillarXY[1]/2-footingXY[0]/2]
    beamY = [beamXZ[0]/2-footingXY[1]/2, beamXZ[0]]
    
    for distance in pillarDisTraslation[1:]:
        beamX.extend([-pillarXY[1], distance - pillarXY[1]])
    
    for distance in pillarDis:
        beamY.extend([beamXZ[0]-distance,beamXZ[0]])
    
    beamZ = [-3-2, beamXZ[1]]
    for disB in beamDis:
        beamZ.extend([beamXZ[1] - disB, beamXZ[1]])
    
    beams = INSR(PROD)([QUOTE(beamX),QUOTE(beamY),QUOTE(beamZ)])

    traslations = STRUCT(finalTraslationList)
    return STRUCT([traslations,beams])
    
VIEW(ggpl_bone_structure("./frame_data_406007.csv"))
                

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000001C98AC60> >

## Immagine Risultato

<img src="./coloreIMG.png">

La parte colorata è il risultato della traslazione. In rosso i pilastri, in giallo le fondamenta, in verde le travi. Le travi in bianco sono quelle aggiunte dopo la traslazione.